# Retail Case Study Data
---
Datos libres usados para el ejercicio:
https://www.kaggle.com/datasets/darpan25bajaj/retail-case-study-data

In [1]:
# import pyspark.pandas as pypd
#El tipo de dataframe que se genera con base en esta función permite distrbuir el procesamiento y aprovechar la ventaja de Spark.
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
spark = SparkSession.builder.appName("3B_Test").getOrCreate()

In [3]:
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# 0.Cargando los datos
Tenemos 3 fuentes de datos:
- Customer.csv: aquí hay información relacionada con los clientes.
- prod_cat_info.csv: este es el catalogo de productos.
- Transactions.csv: aquí estan registradas las ventas y hay campos que se pueden usar para unir esta tabla con las dos anteriores

In [4]:
df_customer = spark.read.csv("Customer.csv",header=True)
df_customer.show(5)

+-----------+----------+------+---------+
|customer_Id|       DOB|Gender|city_code|
+-----------+----------+------+---------+
|     268408|02-01-1970|     M|        4|
|     269696|07-01-1970|     F|        8|
|     268159|08-01-1970|     F|        8|
|     270181|10-01-1970|     F|        2|
|     268073|11-01-1970|     M|        1|
+-----------+----------+------+---------+
only showing top 5 rows



In [5]:
df_prods = spark.read.csv("prod_cat_info.csv",header=True)
df_prods.show(5)

+-------------+--------+-----------------+-----------+
|prod_cat_code|prod_cat|prod_sub_cat_code|prod_subcat|
+-------------+--------+-----------------+-----------+
|            1|Clothing|                4|       Mens|
|            1|Clothing|                1|      Women|
|            1|Clothing|                3|       Kids|
|            2|Footwear|                1|       Mens|
|            2|Footwear|                3|      Women|
+-------------+--------+-----------------+-----------+
only showing top 5 rows



In [6]:
df_transactions = spark.read.csv("Transactions.csv",header=True)
df_transactions.show(5)

+--------------+-------+----------+----------------+-------------+---+-----+-------+---------+----------+
|transaction_id|cust_id| tran_date|prod_subcat_code|prod_cat_code|Qty| Rate|    Tax|total_amt|Store_type|
+--------------+-------+----------+----------------+-------------+---+-----+-------+---------+----------+
|   80712190438| 270351|28-02-2014|               1|            1| -5| -772|  405.3|  -4265.3|    e-Shop|
|   29258453508| 270384|27-02-2014|               5|            3| -5|-1497|785.925|-8270.925|    e-Shop|
|   51750724947| 273420|24-02-2014|               6|            5| -2| -791| 166.11| -1748.11|  TeleShop|
|   93274880719| 271509|24-02-2014|              11|            6| -3|-1363|429.345|-4518.345|    e-Shop|
|   51750724947| 273420|23-02-2014|               6|            5| -2| -791| 166.11| -1748.11|  TeleShop|
+--------------+-------+----------+----------------+-------------+---+-----+-------+---------+----------+
only showing top 5 rows



## 1.Limpieza de Datos

#### 1.1 Valores Nulos

In [7]:
df_customer.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_customer.columns]).show()

+-----------+---+------+---------+
|customer_Id|DOB|Gender|city_code|
+-----------+---+------+---------+
|          0|  0|     2|        2|
+-----------+---+------+---------+



In [8]:
df_customer.where("city_code is null").show()
df_customer.where("Gender is null").show()

+-----------+----------+------+---------+
|customer_Id|       DOB|Gender|city_code|
+-----------+----------+------+---------+
|     268447|14-07-1970|     M|     null|
|     268709|09-09-1970|     F|     null|
+-----------+----------+------+---------+

+-----------+----------+------+---------+
|customer_Id|       DOB|Gender|city_code|
+-----------+----------+------+---------+
|     267199|14-02-1970|  null|        2|
|     271626|02-06-1970|  null|        6|
+-----------+----------+------+---------+



Tenemos nulos:
- en city_code, 2 valores.
- en Gender, 2 valores.

Podremos encontrar esos datos?

In [9]:
df_customer.where("customer_Id in (268447,268709,267199,271626)").show()

+-----------+----------+------+---------+
|customer_Id|       DOB|Gender|city_code|
+-----------+----------+------+---------+
|     267199|14-02-1970|  null|        2|
|     271626|02-06-1970|  null|        6|
|     268447|14-07-1970|     M|     null|
|     268709|09-09-1970|     F|     null|
+-----------+----------+------+---------+



No, los registros en esta tabla son únicos, algo que es lo correcto.
Podría eliminar estos registros nulos, sin embargo, no lo haré pero tendré en cuenta esto al momento de generar futuros análisis. Aún así, los 4 customers si tienen DOB (Date of birth) entonces pueden seguir siendo útiles para otro tipo de análisis.

In [10]:
df_prods.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_prods.columns]).show()
# No hay nulos!

+-------------+--------+-----------------+-----------+
|prod_cat_code|prod_cat|prod_sub_cat_code|prod_subcat|
+-------------+--------+-----------------+-----------+
|            0|       0|                0|          0|
+-------------+--------+-----------------+-----------+



In [11]:
df_transactions.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_transactions.columns]).show()
# No hay nulos!

+--------------+-------+---------+----------------+-------------+---+----+---+---------+----------+
|transaction_id|cust_id|tran_date|prod_subcat_code|prod_cat_code|Qty|Rate|Tax|total_amt|Store_type|
+--------------+-------+---------+----------------+-------------+---+----+---+---------+----------+
|             0|      0|        0|               0|            0|  0|   0|  0|        0|         0|
+--------------+-------+---------+----------------+-------------+---+----+---+---------+----------+



#### 1.2 Tipos de datos

In [12]:
df_customer.printSchema()
# Todo bien, menos el formato de DOB, lo cambiare a un formato de fecha

root
 |-- customer_Id: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- city_code: string (nullable = true)



In [13]:
df_customer.select("DOB").show()
# Transformar a formato estándar: YYYY-MM-DD

+----------+
|       DOB|
+----------+
|02-01-1970|
|07-01-1970|
|08-01-1970|
|10-01-1970|
|11-01-1970|
|15-01-1970|
|15-01-1970|
|16-01-1970|
|18-01-1970|
|21-01-1970|
|22-01-1970|
|23-01-1970|
|25-01-1970|
|26-01-1970|
|29-01-1970|
|29-01-1970|
|29-01-1970|
|01-02-1970|
|01-02-1970|
|01-02-1970|
+----------+
only showing top 20 rows



__1.2.1 Fecha de Nacimiento en Customer__

In [14]:
df_customer = df_customer.withColumn("DOB", to_date(df_customer.DOB, 'dd-MM-yyyy'))

In [15]:
df_customer.show(3)
df_customer.printSchema()

+-----------+----------+------+---------+
|customer_Id|       DOB|Gender|city_code|
+-----------+----------+------+---------+
|     268408|1970-01-02|     M|        4|
|     269696|1970-01-07|     F|        8|
|     268159|1970-01-08|     F|        8|
+-----------+----------+------+---------+
only showing top 3 rows

root
 |-- customer_Id: string (nullable = true)
 |-- DOB: date (nullable = true)
 |-- Gender: string (nullable = true)
 |-- city_code: string (nullable = true)



__1.2.2 Modificando Valores de Fecha de Nacimiento__
- Esto va servir para un análisis posterior.

In [16]:
df_customer = df_customer.withColumn('DOB_New', 
                       when(col('DOB') < '1975-12-15', '1955-06-15')
                       .when(col("DOB") > '1981-06-15', '2007-06-15')
                       .otherwise(df_customer.DOB)
                      )

In [17]:
print(df_customer.agg(min("DOB_New")).head())
print(df_customer.agg(max("DOB_New")).head())

Row(min(DOB_New)='1955-06-15')
Row(max(DOB_New)='2007-06-15')


In [18]:
df_prods.printSchema()
# Bien, todos strings

root
 |-- prod_cat_code: string (nullable = true)
 |-- prod_cat: string (nullable = true)
 |-- prod_sub_cat_code: string (nullable = true)
 |-- prod_subcat: string (nullable = true)



In [19]:
df_transactions.printSchema()
# Igual acá con el formato de fecha para tran_date

root
 |-- transaction_id: string (nullable = true)
 |-- cust_id: string (nullable = true)
 |-- tran_date: string (nullable = true)
 |-- prod_subcat_code: string (nullable = true)
 |-- prod_cat_code: string (nullable = true)
 |-- Qty: string (nullable = true)
 |-- Rate: string (nullable = true)
 |-- Tax: string (nullable = true)
 |-- total_amt: string (nullable = true)
 |-- Store_type: string (nullable = true)



__1.2.3 Fecha de Transacción de Venta__

In [20]:
df_transactions.select("tran_date").show(3)
df_transactions.select("tran_date").distinct().show(3)
# Acá se puede ver como hay distintos formatos para estos valores.

+----------+
| tran_date|
+----------+
|28-02-2014|
|27-02-2014|
|24-02-2014|
+----------+
only showing top 3 rows

+----------+
| tran_date|
+----------+
|27-12-2013|
|14-01-2013|
|  9/1/2013|
+----------+
only showing top 3 rows



In [21]:
df_transactions.show(3)
df_transactions = df_transactions.withColumn("tran_date", regexp_replace(col("tran_date"), "/", "-"))
df_transactions = df_transactions.withColumn("tran_date", to_date(df_transactions.tran_date, 'dd-MM-yyyy'))
df_transactions.show(3)

+--------------+-------+----------+----------------+-------------+---+-----+-------+---------+----------+
|transaction_id|cust_id| tran_date|prod_subcat_code|prod_cat_code|Qty| Rate|    Tax|total_amt|Store_type|
+--------------+-------+----------+----------------+-------------+---+-----+-------+---------+----------+
|   80712190438| 270351|28-02-2014|               1|            1| -5| -772|  405.3|  -4265.3|    e-Shop|
|   29258453508| 270384|27-02-2014|               5|            3| -5|-1497|785.925|-8270.925|    e-Shop|
|   51750724947| 273420|24-02-2014|               6|            5| -2| -791| 166.11| -1748.11|  TeleShop|
+--------------+-------+----------+----------------+-------------+---+-----+-------+---------+----------+
only showing top 3 rows

+--------------+-------+----------+----------------+-------------+---+-----+-------+---------+----------+
|transaction_id|cust_id| tran_date|prod_subcat_code|prod_cat_code|Qty| Rate|    Tax|total_amt|Store_type|
+--------------+-----

#### 1.3 Duplicados

In [22]:
df_dict = {"df_customer":df_customer,
           "df_prods":df_prods,
           "df_transactions":df_transactions
}

In [23]:
counter = 0
for df in [df_customer,df_prods,df_transactions]:
    if df.count() > df.dropDuplicates(df.columns).count():
        print("[WARNING]: Duplicados encontrados en",list(df_dict.keys())[counter], "revisar.")
    else:
        print("No se encontraron registros duplicados en el dataframe", list(df_dict.keys())[counter], "muy bien.")
    counter += 1

No se encontraron registros duplicados en el dataframe df_customer muy bien.
No se encontraron registros duplicados en el dataframe df_prods muy bien.
[WARNING]: Duplicados encontrados en df_transactions revisar.


In [24]:
print(df_transactions.count())
df_transactions = df_transactions.dropDuplicates()
print(df_transactions.count())

23053
23040


## 2. Nuevas Columnas
- Creación de nuevas columnas y transformación de otras.

In [25]:
df_transactions.show(3)
df_prods.show(3)
df_customer.show(3)

+--------------+-------+----------+----------------+-------------+---+----+-------+---------+----------+
|transaction_id|cust_id| tran_date|prod_subcat_code|prod_cat_code|Qty|Rate|    Tax|total_amt|Store_type|
+--------------+-------+----------+----------------+-------------+---+----+-------+---------+----------+
|   40128462174| 273020|2014-02-03|               1|            2|  2|1362| 286.02|  3010.02|    e-Shop|
|   13844622404| 271302|2014-01-11|               3|            1|  2| 923| 193.83|  2039.83|    e-Shop|
|   83307796382| 273016|2013-12-21|               6|            5|  5| 481|252.525| 2657.525|    e-Shop|
+--------------+-------+----------+----------------+-------------+---+----+-------+---------+----------+
only showing top 3 rows

+-------------+--------+-----------------+-----------+
|prod_cat_code|prod_cat|prod_sub_cat_code|prod_subcat|
+-------------+--------+-----------------+-----------+
|            1|Clothing|                4|       Mens|
|            1|Cloth

### 2.1 Creando columna de Edad y Age_Category:
- Utilizare la columna de DOB (fecha de nacimiento) para creear una columna de Edad en la base de datos df_customer.
- Esta columna nueva puede servir para segmentar comportamientos entre jovenes y personas de mediana o mayor edad.

__2.1.1 Age__

In [26]:
df_customer = df_customer.withColumn('Age', (months_between(current_date(), col('DOB_New')) / 12).cast('int'))

In [27]:
df_customer.head(5),df_customer.tail(5)

([Row(customer_Id='268408', DOB=datetime.date(1970, 1, 2), Gender='M', city_code='4', DOB_New='1955-06-15', Age=68),
  Row(customer_Id='269696', DOB=datetime.date(1970, 1, 7), Gender='F', city_code='8', DOB_New='1955-06-15', Age=68),
  Row(customer_Id='268159', DOB=datetime.date(1970, 1, 8), Gender='F', city_code='8', DOB_New='1955-06-15', Age=68),
  Row(customer_Id='270181', DOB=datetime.date(1970, 1, 10), Gender='F', city_code='2', DOB_New='1955-06-15', Age=68),
  Row(customer_Id='268073', DOB=datetime.date(1970, 1, 11), Gender='M', city_code='1', DOB_New='1955-06-15', Age=68)],
 [Row(customer_Id='274474', DOB=datetime.date(1992, 12, 19), Gender='M', city_code='2', DOB_New='2007-06-15', Age=16),
  Row(customer_Id='267666', DOB=datetime.date(1992, 12, 24), Gender='M', city_code='6', DOB_New='2007-06-15', Age=16),
  Row(customer_Id='270476', DOB=datetime.date(1992, 12, 25), Gender='F', city_code='3', DOB_New='2007-06-15', Age=16),
  Row(customer_Id='269626', DOB=datetime.date(1992, 12,

__2.1.2 Age_Category__

In [28]:
df_customer = df_customer.withColumn('Age_Category', 
                       when(col('Age') < 19, "Temprana Edad")
                       .when(col("Age").between(19,60), "Mediana Edad")
                       .otherwise("Vejez")
                      )

In [29]:
df_customer.show(5)

+-----------+----------+------+---------+----------+---+------------+
|customer_Id|       DOB|Gender|city_code|   DOB_New|Age|Age_Category|
+-----------+----------+------+---------+----------+---+------------+
|     268408|1970-01-02|     M|        4|1955-06-15| 68|       Vejez|
|     269696|1970-01-07|     F|        8|1955-06-15| 68|       Vejez|
|     268159|1970-01-08|     F|        8|1955-06-15| 68|       Vejez|
|     270181|1970-01-10|     F|        2|1955-06-15| 68|       Vejez|
|     268073|1970-01-11|     M|        1|1955-06-15| 68|       Vejez|
+-----------+----------+------+---------+----------+---+------------+
only showing top 5 rows



#### 2.1.3 City_Code:
- Asignar valores en texto a estos Ids.

In [33]:
df_customer.select("city_code").distinct().show()
# Tenemos:
# 1,2,3,4,5,6,7,8,9,10

+---------+
|city_code|
+---------+
|        7|
|        3|
|        8|
|     null|
|        5|
|        6|
|        9|
|        1|
|       10|
|        4|
|        2|
+---------+



In [37]:
df_customer = df_customer.withColumn('City_Name', 
                       when(col('city_code') == 1, "Ciudad 1")
                       .when(col("city_code") == 2, "Ciudad 2")
                       .when(col("city_code") == 3, "Ciudad 3")
                       .when(col("city_code") == 4, "Ciudad 4")
                       .when(col("city_code") == 5, "Ciudad 5")
                       .when(col("city_code") == 6, "Ciudad 6")
                       .when(col("city_code") == 7, "Ciudad 7")
                       .when(col("city_code") == 8, "Ciudad 8")
                       .when(col("city_code") == 9, "Ciudad 9")
                       .otherwise("Ciudad 10")
                      )

In [38]:
df_customer.show()

+-----------+----------+------+---------+----------+---+------------+---------+
|customer_Id|       DOB|Gender|city_code|   DOB_New|Age|Age_Category|City_Name|
+-----------+----------+------+---------+----------+---+------------+---------+
|     268408|1970-01-02|     M|        4|1955-06-15| 68|       Vejez| Ciudad 4|
|     269696|1970-01-07|     F|        8|1955-06-15| 68|       Vejez| Ciudad 8|
|     268159|1970-01-08|     F|        8|1955-06-15| 68|       Vejez| Ciudad 8|
|     270181|1970-01-10|     F|        2|1955-06-15| 68|       Vejez| Ciudad 2|
|     268073|1970-01-11|     M|        1|1955-06-15| 68|       Vejez| Ciudad 1|
|     273216|1970-01-15|     F|        5|1955-06-15| 68|       Vejez| Ciudad 5|
|     266929|1970-01-15|     M|        8|1955-06-15| 68|       Vejez| Ciudad 8|
|     275152|1970-01-16|     M|        4|1955-06-15| 68|       Vejez| Ciudad 4|
|     275034|1970-01-18|     F|        4|1955-06-15| 68|       Vejez| Ciudad 4|
|     273966|1970-01-21|     M|        8

## 3. Joins.
- Vamos a crear una 'master table', uniendo las 3 bases de datos que tenemos.
- Lo que más me interesa son los datos de venta, por lo tanto, unire a esta la información que se encuentra en las tablas del catalogo de productos (df_prods) y la información de los clientes (df_customer).
- Así que, los joins van a ser con base a los registros de df_transactions, que es donde estan todos los registros de ventas.

In [39]:
df_transactions.show(2)
df_prods.show(2)
df_customer.show(2)

+--------------+-------+----------+----------------+-------------+---+----+------+---------+----------+
|transaction_id|cust_id| tran_date|prod_subcat_code|prod_cat_code|Qty|Rate|   Tax|total_amt|Store_type|
+--------------+-------+----------+----------------+-------------+---+----+------+---------+----------+
|   40128462174| 273020|2014-02-03|               1|            2|  2|1362|286.02|  3010.02|    e-Shop|
|   13844622404| 271302|2014-01-11|               3|            1|  2| 923|193.83|  2039.83|    e-Shop|
+--------------+-------+----------+----------------+-------------+---+----+------+---------+----------+
only showing top 2 rows

+-------------+--------+-----------------+-----------+
|prod_cat_code|prod_cat|prod_sub_cat_code|prod_subcat|
+-------------+--------+-----------------+-----------+
|            1|Clothing|                4|       Mens|
|            1|Clothing|                1|      Women|
+-------------+--------+-----------------+-----------+
only showing top 2 ro

#### 3.1 Tabla de Ventas con Tabla de Clientes

In [40]:
# Asignamos alias para poder seleccionar solo las columnas deseadas y asi evitar columnas duplicadas.
df_transactions_1 = df_transactions.alias("a").join(df_customer.alias("b"),df_transactions.cust_id == df_customer.customer_Id,"left")\
    .select("a.*","b.Gender","b.city_code","b.Age","b.Age_Category", "b.City_Name")

#### 3.2 Tabla de Ventas+Clientes con Tabla del Catalogo de Productos

In [42]:
df_transactions_2 = df_transactions_1.alias("a").join(df_prods.alias("b"),df_transactions.prod_cat_code == df_prods.prod_cat_code,"left")\
    .select("a.*","b.prod_cat","b.prod_subcat")

In [43]:
df_transactions_2.printSchema()
# Aca se ven las columnas integradas.

root
 |-- transaction_id: string (nullable = true)
 |-- cust_id: string (nullable = true)
 |-- tran_date: date (nullable = true)
 |-- prod_subcat_code: string (nullable = true)
 |-- prod_cat_code: string (nullable = true)
 |-- Qty: string (nullable = true)
 |-- Rate: string (nullable = true)
 |-- Tax: string (nullable = true)
 |-- total_amt: string (nullable = true)
 |-- Store_type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- city_code: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Age_Category: string (nullable = true)
 |-- City_Name: string (nullable = true)
 |-- prod_cat: string (nullable = true)
 |-- prod_subcat: string (nullable = true)



__3.3 Después de los joins, hay nulos?__

In [44]:
df_transactions_2.select([count(when(col(c).isNull(), c)).alias(c) for c in df_transactions_2.columns]).show()

+--------------+-------+---------+----------------+-------------+---+----+---+---------+----------+------+---------+---+------------+---------+--------+-----------+
|transaction_id|cust_id|tran_date|prod_subcat_code|prod_cat_code|Qty|Rate|Tax|total_amt|Store_type|Gender|city_code|Age|Age_Category|City_Name|prod_cat|prod_subcat|
+--------------+-------+---------+----------------+-------------+---+----+---+---------+----------+------+---------+---+------------+---------+--------+-----------+
|             0|      0|        0|               0|            0|  0|   0|  0|        0|         0|    40|       36|  0|           0|        0|       0|          0|
+--------------+-------+---------+----------------+-------------+---+----+---+---------+----------+------+---------+---+------------+---------+--------+-----------+



Nulos en city_code y Gender, como en un principio vimos. Ahora si eliminaremos esos registros para no crear problemas al momento de crear agrupaciones y el archivo final para el dashboard.

In [45]:
print(df_transactions_2.count())
df_transactions_2 = df_transactions_2.na.drop()
print(df_transactions_2.count())

99237
99161


Listo! Tenemos una master table. Ahora podemos realizar queries para encontrar cosas interesantes.

## 4. Análisis:
- Esta sección será dedicada a principalmente realizar agrupaciones y categorizaciones para generar conocimiento con base en la información que tenemos.

### 4.0 Master Table final filtrada

In [46]:
df_transactions_f = df_transactions_2[['tran_date','cust_id','prod_cat','prod_subcat','Qty','Rate','Tax','total_amt','Store_type','Gender',
                  'city_code','Age','Age_Category', 'City_Name']]

In [47]:
df_transactions_f.show(5)

+----------+-------+--------+-----------+---+----+------+---------+----------+------+---------+---+-------------+---------+
| tran_date|cust_id|prod_cat|prod_subcat|Qty|Rate|   Tax|total_amt|Store_type|Gender|city_code|Age| Age_Category|City_Name|
+----------+-------+--------+-----------+---+----+------+---------+----------+------+---------+---+-------------+---------+
|2014-02-03| 273020|Footwear|       Kids|  2|1362|286.02|  3010.02|    e-Shop|     M|        5| 16|Temprana Edad| Ciudad 5|
|2014-02-03| 273020|Footwear|      Women|  2|1362|286.02|  3010.02|    e-Shop|     M|        5| 16|Temprana Edad| Ciudad 5|
|2014-02-03| 273020|Footwear|       Mens|  2|1362|286.02|  3010.02|    e-Shop|     M|        5| 16|Temprana Edad| Ciudad 5|
|2014-01-11| 271302|Clothing|       Kids|  2| 923|193.83|  2039.83|    e-Shop|     M|        1| 16|Temprana Edad| Ciudad 1|
|2014-01-11| 271302|Clothing|      Women|  2| 923|193.83|  2039.83|    e-Shop|     M|        1| 16|Temprana Edad| Ciudad 1|
+-------

### 4.1 Ventas Totales Mensuales

In [48]:
df_transactions_f = df_transactions_f.withColumn("YM", date_format(to_date(df_transactions_f.tran_date, 'yyyy-MM-dd'),'yyyy-MM'))
df_transactions_f.show(2)

+----------+-------+--------+-----------+---+----+------+---------+----------+------+---------+---+-------------+---------+-------+
| tran_date|cust_id|prod_cat|prod_subcat|Qty|Rate|   Tax|total_amt|Store_type|Gender|city_code|Age| Age_Category|City_Name|     YM|
+----------+-------+--------+-----------+---+----+------+---------+----------+------+---------+---+-------------+---------+-------+
|2014-02-03| 273020|Footwear|       Kids|  2|1362|286.02|  3010.02|    e-Shop|     M|        5| 16|Temprana Edad| Ciudad 5|2014-02|
|2014-02-03| 273020|Footwear|      Women|  2|1362|286.02|  3010.02|    e-Shop|     M|        5| 16|Temprana Edad| Ciudad 5|2014-02|
+----------+-------+--------+-----------+---+----+------+---------+----------+------+---------+---+-------------+---------+-------+
only showing top 2 rows



In [49]:
df_transactions_f.createTempView('Master')

In [50]:
query = "SELECT YM as Fecha, round(sum(total_amt)) as Ventas from Master\
        GROUP BY Fecha\
        ORDER BY Fecha"

In [51]:
spark.sql(query).show()

+-------+---------+
|  Fecha|   Ventas|
+-------+---------+
|2011-01|1530794.0|
|2011-02|4943032.0|
|2011-03|5985670.0|
|2011-04|5964267.0|
|2011-05|5141492.0|
|2011-06|5306149.0|
|2011-07|5639165.0|
|2011-08|5200621.0|
|2011-09|6234628.0|
|2011-10|6089592.0|
|2011-11|6241230.0|
|2011-12|5726659.0|
|2012-01|5325797.0|
|2012-02|5487791.0|
|2012-03|6202447.0|
|2012-04|5262395.0|
|2012-05|5891762.0|
|2012-06|5644567.0|
|2012-07|5437883.0|
|2012-08|5867489.0|
+-------+---------+
only showing top 20 rows



- Podemos ver que los datos son desde julio 2012 hasta febrero 2014.
- En general las ventas mensuales tienen normalidad, a excepeción de este punto:
    - 2011-01: ni siquiera se llego a vender 2 millones, todos los demás meses se vende más de 4.9 millones.

### 4.2 Ventas Totales por distintas agregaciones

#### 4.2.1 Categorías y Subcategorías

In [52]:
query = "SELECT prod_cat as Categoria, round(sum(total_amt)) as Ventas from Master\
        GROUP BY Categoria\
        ORDER BY Ventas DESC"

spark.sql(query).show(25)

+----------------+-----------+
|       Categoria|     Ventas|
+----------------+-----------+
|           Books|7.6917746E7|
|     Electronics|5.3600821E7|
|Home and kitchen|3.3742992E7|
|        Clothing|1.8752491E7|
|        Footwear|1.8670067E7|
|            Bags|  8252158.0|
+----------------+-----------+



In [53]:
query = "SELECT prod_cat as Categoria, prod_subcat as Subcategoria, round(sum(total_amt)) as Ventas from Master\
        GROUP BY Categoria, Subcategoria\
        ORDER BY Ventas DESC"

spark.sql(query).show(25)

+----------------+-------------------+-----------+
|       Categoria|       Subcategoria|     Ventas|
+----------------+-------------------+-----------+
|           Books|           Children|1.2819624E7|
|           Books|            Fiction|1.2819624E7|
|           Books|                DIY|1.2819624E7|
|           Books|             Comics|1.2819624E7|
|           Books|        Non-Fiction|1.2819624E7|
|           Books|           Academic|1.2819624E7|
|     Electronics|            Cameras|1.0720164E7|
|     Electronics|    Audio and video|1.0720164E7|
|     Electronics|            Mobiles|1.0720164E7|
|     Electronics|          Computers|1.0720164E7|
|     Electronics|Personal Appliances|1.0720164E7|
|Home and kitchen|              Tools|  8435748.0|
|Home and kitchen|         Furnishing|  8435748.0|
|Home and kitchen|            Kitchen|  8435748.0|
|Home and kitchen|               Bath|  8435748.0|
|        Clothing|               Mens|  6250830.0|
|        Clothing|             

In [54]:
query = "SELECT round(sum(total_amt)) as Ventas from Master\
        WHERE prod_cat = 'Books' AND prod_subcat = 'Children'"
spark.sql(query).show()

+-----------+
|     Ventas|
+-----------+
|1.2819624E7|
+-----------+



In [55]:
query = "SELECT round(sum(total_amt)) as Ventas from Master\
        WHERE prod_cat = 'Books' AND prod_subcat = 'Fiction'"
spark.sql(query).show()

+-----------+
|     Ventas|
+-----------+
|1.2819624E7|
+-----------+



- La categoría de Libros es la que tiene más ventas en todas sus subcategorías también.
- Seguida de la categoría de Electrónica.
- En último lugar tenemos a las Bolsas.

#### 4.2.2 Tipo de tienda

In [56]:
query = "SELECT Store_type as Tipo_de_Comercio, round(sum(total_amt)) as Ventas from Master\
        GROUP BY Tipo_de_Comercio\
        ORDER BY Ventas DESC"

spark.sql(query).show(25)

+----------------+-----------+
|Tipo_de_Comercio|     Ventas|
+----------------+-----------+
|          e-Shop| 8.597055E7|
|  Flagship store|4.1914909E7|
|             MBR|4.1667139E7|
|        TeleShop|4.0383676E7|
+----------------+-----------+



- La mayoría de ventas ocurren en el esquema en línea.

#### 4.2.3 Genero y Edades

¿Quién compra más las mujeres o los hombres?

In [57]:
query = "SELECT Gender as Genero, round(sum(total_amt)) as Ventas from Master\
        GROUP BY Genero\
        ORDER BY Ventas DESC"

spark.sql(query).show(25)

+------+------------+
|Genero|      Ventas|
+------+------------+
|     M|1.08433503E8|
|     F|1.01502771E8|
+------+------------+



- Casi lo mismo! Aunque esta vez los hombres ganan.

¿Y dentro de los generos, quienes compran más? ¿Adultos, jóvenes o mediana edad?

In [58]:
query = "SELECT Gender as Genero, Age_Category as Edad, round(sum(total_amt)) as Ventas from Master\
        GROUP BY Genero, Edad\
        ORDER BY Ventas DESC"

spark.sql(query).show(25)

+------+-------------+-----------+
|Genero|         Edad|     Ventas|
+------+-------------+-----------+
|     M|Temprana Edad|5.3693775E7|
|     F|Temprana Edad|5.0780957E7|
|     M|        Vejez|2.8382889E7|
|     M| Mediana Edad|2.6356839E7|
|     F|        Vejez|2.5969996E7|
|     F| Mediana Edad|2.4751818E7|
+------+-------------+-----------+



- Sin importar el género, los jovenes de edad son los mejores compradores para este retail.

#### 4.2.4 Ciudad:
- Idealmente se tendría que haber usado no el código si no el nombre de las ciudades, pero no vienen desde el origen de los datos.

In [59]:
query = "SELECT City_Name as Ciudad, round(sum(total_amt)) as Ventas from Master\
        GROUP BY Ciudad\
        ORDER BY Ventas DESC"

spark.sql(query).show(25)

+---------+-----------+
|   Ciudad|     Ventas|
+---------+-----------+
| Ciudad 3|2.2744245E7|
| Ciudad 4|2.2361621E7|
| Ciudad 5|2.1457588E7|
| Ciudad 7|2.1360346E7|
| Ciudad 2|2.1104847E7|
|Ciudad 10| 2.085613E7|
| Ciudad 8|2.0621868E7|
| Ciudad 1|2.0558143E7|
| Ciudad 9|2.0034979E7|
| Ciudad 6|1.8836506E7|
+---------+-----------+



- La ciudad 3 es la que más ventas tiene.

Nota:
- Importante también obtener este tipo de métricas con base a Unidades Vendidas y no solo Ventas.

---
---
---
Exportando datos a Excel para el dashboard.

In [60]:
df_transactions_f.show()

+----------+-------+----------------+-------------------+---+-----+-------+---------+----------+------+---------+---+-------------+---------+-------+
| tran_date|cust_id|        prod_cat|        prod_subcat|Qty| Rate|    Tax|total_amt|Store_type|Gender|city_code|Age| Age_Category|City_Name|     YM|
+----------+-------+----------------+-------------------+---+-----+-------+---------+----------+------+---------+---+-------------+---------+-------+
|2014-02-03| 273020|        Footwear|               Kids|  2| 1362| 286.02|  3010.02|    e-Shop|     M|        5| 16|Temprana Edad| Ciudad 5|2014-02|
|2014-02-03| 273020|        Footwear|              Women|  2| 1362| 286.02|  3010.02|    e-Shop|     M|        5| 16|Temprana Edad| Ciudad 5|2014-02|
|2014-02-03| 273020|        Footwear|               Mens|  2| 1362| 286.02|  3010.02|    e-Shop|     M|        5| 16|Temprana Edad| Ciudad 5|2014-02|
|2014-01-11| 271302|        Clothing|               Kids|  2|  923| 193.83|  2039.83|    e-Shop|    

In [61]:
import pandas as pd

In [62]:
# Converting spark dataframe to pandas dataframe
pandas_df = df_transactions_f.toPandas()

In [63]:
pandas_df.shape

(99161, 15)

In [64]:
# Exporting pandas dataframe to xlsx file
pandas_df.to_excel('3B_test.xlsx', index=False)